In [ ]:
#pip3 --python=`which python` install .
import flowsig as fs
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt

sc.settings.set_figure_params(dpi=300)
sc.settings.figdir = '/data/project/AI4Omic/MASLD/results/scRNA/flowsig'

2025-09-22 19:21:09.670229: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-22 19:21:09.746759: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-22 19:21:10.177375: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-22 19:21:11.257352: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


CCC_control = pd.read_csv('/data/project/AI4Omic/MASLD/results/scRNA/CellphoneDB/statistical_analysis_significant_means_Control.txt', index_col=0, sep='\t')
CCC_control.dropna(subset=['gene_a', 'gene_b'], inplace=True)
CCC_control = CCC_control[CCC_control['secreted'] == True]
CCC_control = CCC_control[(CCC_control['receptor_a'] == False) & (CCC_control['receptor_b'] == True)]
CCC_control.rename(columns={'classification': 'pathway_name', 'annotation_strategy': 'evidence', 'gene_a': 'ligand', 'gene_b': 'receptor'}, inplace=True)

In [ ]:
# Load the data and cell-cell communication inference
adata = sc.read_h5ad('/data/project/AI4Omic/MASLD/results/scRNA/CellphoneDB/adata_major_29samples.h5ad')
condition_key = 'disease_status'

# Load the cell-cell communication inference
cellchat_ctrl = pd.read_csv('/data/project/AI4Omic/MASLD/results/scRNA/flowsig/cellchat_communications_ctrl.csv', index_col=0)
cellchat_masl = pd.read_csv('/data/project/AI4Omic/MASLD/results/scRNA/flowsig/cellchat_communications_masl.csv', index_col=0)
cellchat_mash = pd.read_csv('/data/project/AI4Omic/MASLD/results/scRNA/flowsig/cellchat_communications_mash.csv', index_col=0)


#adata = adata[adata.obs[condition_key].isin(['CTRL', 'MASL'])].copy()
#adata.uns['cellchat_output'] = { 'CTRL': cellchat_ctrl, 'MASL': cellchat_masl}

adata = adata[adata.obs[condition_key].isin(['MASH', 'MASL'])].copy()
adata.uns['cellchat_output'] = { 'MASL': cellchat_masl, 'MASH': cellchat_mash}

In [3]:
import re
adata = adata[:, ~adata.var_names.str.startswith(('MT-', 'RPL', 'RPS'))].copy()
pattern = r"^(AC|AL|AP)[0-9A-Z\-]+\.\d+$"
to_remove = [g for g in adata.var_names if re.match(pattern, g)]
print(f"Removed {len(to_remove)} unannotated or noncoding genes.")
adata = adata[:, ~adata.var_names.isin(to_remove)].copy()

#adata = adata[:, mask].copy()
# !pip --python=`which python` install scikit-misc for flavor='seurat_v3'
#sc.pp.highly_variable_genes(adata, n_top_genes=20000, inplace=True, batch_key='sample', flavor='seurat_v3')
#adata = adata[:, adata.var.highly_variable].copy()

Removed 9376 unannotated or noncoding genes.


In [4]:
import numpy as np
len(np.unique(adata.obs['sample']))

23

In [5]:
# Construct GEMs from the raw count data by the iNMF algorithm by pyliger
adata.var.index.name = "gene"
adata.obs.index.name = "cell"
fs.pp.construct_gems_using_pyliger(adata, n_gems=30, layer_key='counts', condition_key=condition_key)

Removing 3954 genes not expressing in MASL.
Removing 2743 genes not expressing in MASH.


100%|██████████| 30/30 [29:31<00:00, 59.05s/it]


# Construct the flow expression matrices

### We construct augmented flow expression matrices for each condition that measure three types of variables:

1. Intercellular signal inflow, i.e., how much of a signal did a cell receive. For non-spatial scRNA-seq, signal inflow is defined as receptor gene expression weighted by the average expression of immediate downstream transcription factors that indicate signal activation.
2. GEMs, which encapsulate intracellular information processing. We define these as cellwise membership to the GEM.
Intercellular signal outflow, i.e., how much of a signal did a cell send. These are simply ligand gene expression.
3. The kay assumption of flowsig is that all intercellular information flows are directed from signal inflows to GEMs, from one GEM to another GEM, and from GEMs to signal outflows.

For non-spatial scRNA-seq, we need to specify the model organism so that FlowSig knows which receptor-transcription factor targets list to look at.

help(fs.pp.construct_flows_from_cellphonedb)
fs.pp.construct_flows_from_cellphonedb(adata,
                                cellphonedb_output_key = ,                                
                                gem_expr_key = 'X_gem',
                                cellphonedb_tfs_key = '',
                                scale_gem_expr = True,
                                model_organism = 'human',
                                flowsig_network_key = 'flowsig_network',
                                flowsig_expr_key = 'X_flow'
                                )

In [6]:
fs.pp.construct_flows_from_cellchat(adata,
                                cellchat_output_key='cellchat_output',                         
                                gem_expr_key='X_gem',
                                scale_gem_expr = True,
                                model_organism = 'human',
                                flowsig_network_key = 'flowsig_network',
                                flowsig_expr_key = 'X_flow'
                                )

### To reduce the number of variables over which we have to infer intercellular flows—and thus computation time—and to prioritise 'informative variables',

- we only retain inflow and outflow signals that are sufficiently differentially flowing between the control and perturbed conditions. 

- We determine differentially flowing signals using a Wilcoxon rank-sum test and retain variables only if they are below a specified adjusted p-value threshold (q-value) and above a specified log-fold-change threshold.

In [39]:
fs.pp.determine_informative_variables(adata,  
                                    flowsig_expr_key = 'X_flow',
                                    flowsig_network_key = 'flowsig_network',
                                    spatial= False,
                                    condition_key = condition_key,
                                    control_key = 'MASL',  # CTRL MASL
                                    qval_threshold = 0.01,
                                    logfc_threshold= 0.8 # 0.8 for MASH versus MASL
                                    )

... storing 'Type' as categorical
... storing 'Downstream_TF' as categorical
... storing 'Type' as categorical
... storing 'Downstream_TF' as categorical


#### If you want to visualise which variables remained, e.g., which are differentially outflowing, you can run the following code:

In [ ]:
from adjustText import adjust_text

for disease_status in ['MASH']: # MASH  MASL
    fig, ax = plt.subplots(figsize=(10, 8))
    scatter = fs.pl.plot_differentially_flowing_signals(adata,
                                            condition_key = condition_key,
                                            pert_key = disease_status,
                                            var_type = 'inflow',
                                            flowsig_expr_key = 'X_flow_orig',
                                            flowsig_network_key = 'flowsig_network_orig',
                                            qval_threshold = 0.01,
                                            logfc_threshold = 0.8,
                                            label_lowqval = True,
                                            scatter_size = 60,
                                            ax=ax,
                                            )
    texts = [child for child in ax.get_children() if isinstance(child, plt.Text)]
    adjust_text(texts, ax=ax)
    plt.tight_layout()
    plt.grid(False)
    plt.savefig(f'/data/project/AI4Omic/MASLD/results/scRNA/flowsig/flowsig_inflow_{disease_status}.png', dpi=600)

... storing 'Type' as categorical
... storing 'Downstream_TF' as categorical


In [18]:
adata.uns['flowsig_network']

{'flow_var_info':                  Type                                      Downstream_TF  \
 WNT10A        outflow                                                      
 EREG          outflow                                                      
 CXCL1         outflow                                                      
 CXCL2         outflow                                                      
 CXCL3         outflow                                                      
 CXCL6         outflow                                                      
 CXCL8         outflow                                                      
 CXCL9         outflow                                                      
 IL1A          outflow                                                      
 IL33          outflow                                                      
 TNFSF15       outflow                                                      
 SPP1          outflow                                     

In [ ]:
for disease_status in ['MASH']: # MASL, MASH
    fig, ax = plt.subplots(figsize=(10, 8))
    fs.pl.plot_differentially_flowing_signals(adata,
                                            condition_key = condition_key,
                                            pert_key = disease_status,
                                            var_type = 'outflow',
                                            flowsig_expr_key = 'X_flow_orig',
                                            flowsig_network_key = 'flowsig_network_orig',
                                            qval_threshold = 0.01,
                                            logfc_threshold = 1.0,
                                            label_lowqval = True,
                                            ax=ax,
                                            scatter_size = 60,
                                            )
    texts = [child for child in ax.get_children() if isinstance(child, plt.Text)]
    adjust_text(texts, ax=ax)
    plt.tight_layout()
    plt.grid(False)
    plt.savefig(f'/data/project/AI4Omic/MASLD/results/scRNA/flowsig/flowsig_outflow_{disease_status}.png', dpi=600, bbox_inches='tight')

... storing 'Type' as categorical
... storing 'Downstream_TF' as categorical


### Learn intercellular flows
We are now in a position to learn the intercellular flows. To increase reliability of objects, we bootstrap aggregate results over a number of realisations. For non-spatial data, we have to specify the condition label and the control condition.

This step uses UT-IGSP to learn what is called a completed partially directed acyclic graph (CPDAG), which encodes directed arcs and undirected edges that describe the Markov Equivalence Class of statistical dependence relations that were learned directly from the data using conditional independence testing (how do variables depend on one another) and conditional invariance testing (which variables changed significantly between conditions). For both tests, we use a parametric partial-correlation-based method. The main reason we used these tests were because they take the least time to run compared to nonparametric kernel-based tests. Any test like the Hilbert-Schmidt Independence Criterion takes way too long for even 10-20 variables. The big caveat is that partial correlation assumes the data is described by a linear Gaussian model, which obviously isn't true for scRNA-seq. It's a long-term goal to add different types of nonparametric conditional independence/invariance tests that can be run in a reasonable amount of time.

In [40]:
fs.tl.learn_intercellular_flows(adata,
                        condition_key = condition_key,
                        control_key = 'MASL',  #CTRL, MASL
                        flowsig_key = 'flowsig_network',
                        flow_expr_key = 'X_flow',
                        use_spatial = False,
                        n_jobs = 50,
                        n_bootstraps = 500)

starting computations on 50 cores


2025-07-02 15:14:29.425284: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-02 15:14:29.427611: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-02 15:14:29.472118: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-02 15:14:30.274913: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-07-02 15:14:31.477946: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see sligh

elapsed time: 796.1604704409838


### Partially validate intercellular flow network
Finally, we will remove any "false positive" edges. Noting that the CPDAG contains directed arcs and undirected arcs we do two things.

First, we remove directed arcs that are not oriented from signal inflow to GEM, GEM to GEM, or from GEM to signal outflow and for undirected edges, we reorient them so that they obey the previous directionalities.

In [41]:
fs.tl.apply_biological_flow(adata,
                            flowsig_network_key = 'flowsig_network',
                            adjacency_key = 'adjacency',
                            validated_key = 'validated')

Second, we will remove directed arcs whose bootstrapped frequencies are below a specified edge threshold as well as undirected edges whose total bootstrapped frequencies are below the same threshold.

In [42]:
#Every time we apply these steps, we generate a new adjacency matrix that describes the intercellular flow network edges. 
# The original output from learn_intercellular_flows is stored in adata.uns['flowsig_network]['network']['adjacency']. 
# After validation using apply_biological_flow, we add the _validated key to the adjacency key, so that the new adjacency is stored in adata.uns['flowsig_network]['network']['adjacency_validated']. 
# After filtering low-confidence edges, the adjacency is stored under adata.uns['flowsig_network]['network']['adjacency_validated_filtered']. 
# As a note, you could change the order of these two post-processing steps, which would mean that the final adjacency would be under the key adjacency_filtered_validated. The results should be similar but it's worth remembering this.
edge_threshold = 0.8
fs.tl.filter_low_confidence_edges(adata,
                                edge_threshold = edge_threshold,
                                flowsig_network_key = 'flowsig_network',
                                adjacency_key = 'adjacency_validated',
                                filtered_key = 'filtered')

[[0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.514 0.526 0.502]
 [0.    0.    0.    ... 0.    0.064 0.066]
 ...
 [0.    0.476 0.    ... 0.    0.    0.42 ]
 [0.    0.496 0.928 ... 0.    0.    0.   ]
 [0.    0.542 0.934 ... 0.496 0.    0.   ]]


In [ ]:
# Save and reload the flowsig outputs
#adata.write('/data/project/AI4Omic/MASLD/results/scRNA/flowsig/adata_MASL_flowsig.h5ad')
#adata = sc.read_h5ad('/data/project/AI4Omic/MASLD/results/scRNA/flowsig/adata_MASL_flowsig.h5ad')
#adata.write('/data/project/AI4Omic/MASLD/results/scRNA/flowsig/adata_MASH_flowsig.h5ad')
adata = sc.read_h5ad('/data/project/AI4Omic/MASLD/results/scRNA/flowsig/adata_MASH_flowsig.h5ad')

In [ ]:
# GEMs explained by celltypes and conditions (disease status)
marker = [f'GEM-{i}' for i in range(1, adata.obsm['X_gem'].shape[1]+1)]
gem_df = pd.DataFrame(adata.obsm['X_gem'], columns=marker)

gem_df.index = adata.obs.index
adata.obs = adata.obs.join(gem_df)

sc.pl.dotplot(adata, var_names=marker, groupby='disease_status', save="GEM_Condition_MASH_versus_MASL.pdf")
#sc.pl.dotplot(adata, var_names=marker, groupby='disease_status', save="GEM_Condition_MASL_versus_CTRL.pdf")
#sc.pl.heatmap(adata, var_names=marker, groupby='disease_status', cmap='Reds', vmax=0.10, vmin=0, save="Condition.pdf")

sc.pl.dotplot(adata, var_names=marker, groupby='cell_type_lvl2', save="GEM_celltype_MASH_versus_MASL.pdf", dendrogram=True)
#sc.pl.dotplot(adata, var_names=marker, groupby='cell_type_lvl2', save="GEM_celltype_MASL_versus_CTRL.pdf", dendrogram=True)
#sc.pl.heatmap(adata, var_names=marker, groupby='cell_type_lvl2', cmap='Reds', vmax=0.10, vmin=0, save="celltype.pdf")

We also note that if you want to explore the network directly, we included a function to generate the directed NetworkX DiGraph object. You will need to generate this for any of the plotting functions.

In [43]:
flow_network = fs.tl.construct_intercellular_flow_network(adata, flowsig_network_key = 'flowsig_network', adjacency_key = 'adjacency_validated_filtered')

In [44]:
flow_var_info = adata.uns['flowsig_network']['flow_var_info']
inflow_vars = flow_var_info.index[flow_var_info['Type'] == 'inflow'].tolist()
print(inflow_vars)
inflow_vars_filtered = [node for node in inflow_vars if node in flow_network.nodes]
print("Filtered inflow_vars:", inflow_vars_filtered)

['CALCRL', 'CSF1R', 'IL13RA2', 'IL1R2', 'IL4R+IL13RA1', 'LIFR+IL6ST', 'LRP1', 'NRP2+PLXNA1']
Filtered inflow_vars: ['CALCRL', 'CSF1R', 'IL13RA2', 'IL1R2', 'IL4R+IL13RA1', 'LIFR+IL6ST', 'LRP1', 'NRP2+PLXNA1']


In [47]:
flow_vars = list(flow_network.nodes())
flow_var_info = adata.uns['flowsig_network']['flow_var_info']
[node for node in flow_vars if flow_var_info.loc[node]['Type'] == 'outflow']
[node for node in flow_vars if flow_var_info.loc[node]['Type'] == 'inflow']

['IL1R2',
 'IL4R+IL13RA1',
 'IL13RA2',
 'LIFR+IL6ST',
 'NRP2+PLXNA1',
 'CSF1R',
 'LRP1',
 'CALCRL']

In [33]:
# 查看HGF的前继节点
for outflow_var in ['HGF', 'IL1A', 'CXCL9', 'WNT10A', 'CXCL3']:
    print(f"Predecessors of {outflow_var}:")
    print(list(flow_network.predecessors(outflow_var)))
for inflow_var in ['LRP1', 'NRP2+PLXNA1']:
    print(f"Successors of {inflow_var}:")
    print(list(flow_network.successors(inflow_var)))
for GEM in ['GEM-13', 'GEM-17', 'GEM-10']:
    print(f"Predecessors of {GEM}:")
    print(list(flow_network.predecessors(GEM)))
    print(f"Successors of {GEM}:")
    print(list(flow_network.successors(GEM)))

Predecessors of HGF:
['GEM-30', 'GEM-20', 'GEM-24', 'GEM-28', 'GEM-10', 'GEM-25', 'GEM-23', 'GEM-13', 'GEM-5', 'GEM-29']
Predecessors of IL1A:
['GEM-14', 'GEM-12', 'GEM-18', 'GEM-23', 'GEM-5', 'GEM-29', 'GEM-17', 'GEM-13', 'GEM-8', 'GEM-30']
Predecessors of CXCL9:
['GEM-5', 'GEM-30', 'GEM-13', 'GEM-28', 'GEM-29', 'GEM-20', 'GEM-16', 'GEM-11']
Predecessors of WNT10A:
['GEM-10', 'GEM-17']
Predecessors of CXCL3:
['GEM-10', 'GEM-17', 'GEM-16', 'GEM-21', 'GEM-24']
Successors of LRP1:
['GEM-14', 'GEM-8', 'GEM-5', 'GEM-13', 'GEM-28', 'GEM-29', 'GEM-30', 'GEM-20', 'GEM-12']
Successors of NRP2+PLXNA1:
['GEM-29', 'GEM-12', 'GEM-14', 'GEM-21', 'GEM-16', 'GEM-8', 'GEM-24']
Predecessors of GEM-13:
['GEM-17', 'GEM-3', 'GEM-5', 'GEM-15', 'GEM-27', 'GEM-2', 'GEM-14', 'GEM-26', 'GEM-30', 'CALCRL', 'GEM-21', 'GEM-20', 'GEM-11', 'GEM-23', 'LRP1', 'LIFR+IL6ST', 'GEM-10', 'GEM-6', 'GEM-28', 'GEM-29', 'GEM-18', 'GEM-8']
Successors of GEM-13:
['GEM-17', 'GEM-3', 'GEM-5', 'GEM-15', 'CXCL9', 'GEM-27', 'GEM-2',

In [ ]:
# glocal intercellular flow plot for MASL versus CTRL
fig, ax = plt.subplots(figsize=(12, 8))
fs.pl.plot_intercellular_flows(adata,
                                flow_network=flow_network,
                                flowsig_network_key='flowsig_network',
                                inflow_vars=['CXCR3', 'KDR', 'LIFR+IL6ST', 'IL13RA2', 'CALCRL'],
                                outflow_vars=['MIF', 'IL33', 'ANGPTL1'],
                                module_vars=['GEM-1', 'GEM-4', 'GEM-5', 'GEM-11', 'GEM-14', 'GEM-21', 'GEM-19', 'GEM-22', 'GEM-24'],
                                align_mode='horizontal',
                                width_scale=1.0,
                                x_margin_offset=0.0,
                                y_margin_offset=0.0,
                                ax=ax
                                )
plt.tight_layout()
plt.grid(False)
plt.savefig('/data/project/AI4Omic/MASLD/results/scRNA/flowsig/flowsig_intercellular_flow_MASL_versus_CTRL.pdf', dpi=600, transparent=True, bbox_inches='tight')

In [ ]:
# glocal intercellular flow plot for MASH versus MASL
fig, ax = plt.subplots(figsize=(12, 8))
fs.pl.plot_intercellular_flows(adata,
                                flow_network=flow_network,
                                flowsig_network_key='flowsig_network',
                                inflow_vars=['LRP1', 'NRP2+PLXNA1', 'IL4R+IL13RA1'], # 
                                outflow_vars=['CXCL3', 'HGF', 'IL1A',  'WNT10A', 'CXCL9'],
                                module_vars=['GEM-5', 'GEM-12', 'GEM-14', 'GEM-13', 'GEM-29', 'GEM-15', 'GEM-30', 'GEM-28', 'GEM-1', 'GEM-3'],
                                align_mode='horizontal',
                                width_scale=1.0,
                                x_margin_offset=0.0,
                                y_margin_offset=0.0,
                                ax=ax
                                )
plt.tight_layout()
plt.grid(False)
plt.savefig('/data/project/AI4Omic/MASLD/results/scRNA/flowsig/flowsig_intercellular_flow_MASH_versus_MASL.pdf', dpi=600, transparent=True, bbox_inches='tight')

In [ ]:
sc.pl.dotplot(adata_ref, ['HGF', "CXCL3", 'WNT10A', 'IL1A', 'CXCL9', 'ANGPTL1', 'IL33', 'MIF'], groupby='cell_type_lvl2', save='_outflow_signals', swap_axes=True, cmap='Blues', vmax=1.0)